# Best clustering

## Load Data

In [1]:
import json
from typing import List
from pathlib import Path
import numpy as np

data_dir = Path('/mnt/data/factcheck/tweets/similarity_web_data')

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
all_to_vecnos = load_dict(data_dir / 'all_vecno.json')
claims = list(all_to_text.values())

print(len(claims))

203057


## Create Groups and Supporting Data

In [2]:
import pandas as pd
from itertools import chain
from collections import Counter

CLUSTERS = 15000

out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')

with open(out_path / f'kmeans_b0_{CLUSTERS}.npy', 'rb') as f:
    clustered_indices_all = np.load(f, allow_pickle=False)

groups = [(clustered_indices_all==group_no).nonzero()[0] 
          for group_no in range(CLUSTERS)]
groups_claims = [[claims[idx] for idx in group] for group in groups]
groups_lens = [len(group) for group in groups]
print('number of empty groups:', (np.array(groups_lens)==0).sum())

number of empty groups: 159


## Filter Out Empty Groups

In [3]:
groups = [g for g, l in zip(groups, groups_lens) if l > 0]
groups_claims = [c for c, l in zip(groups_claims, groups_lens) if l > 0]
groups_lens = [l for l in groups_lens if l > 0]

len(groups), len(groups_claims), len(groups_lens)
groups_lens = np.array(groups_lens)
(groups_lens==0).sum()

0

## Lemmatize

In [4]:
lemma_path = Path('/mnt/data/factcheck/tweets/clustering/lemmatized_all.json')
with open(lemma_path, 'rt') as f:
    lemmas = json.load(f)
    
csv = pd.read_csv('/mnt/data/factcheck/tweets/clustering/syn2015_word_utf8.csv',
                 sep=';', header=None)
source = csv[1].tolist()
most_frequent = source[:300] + ['the', 'and']

def get_main_lemmas(indices):
    used_lemmas = [lemmas[i] for i in indices]
    flattened = list(chain.from_iterable(used_lemmas))
    filtered = [w for w in flattened if len(w) > 2]
    counter = Counter(filtered)
    most_common = counter.most_common(10)
    fmc = [item[0] for item in most_common 
           if item[1]>=10 and item[0] not in most_frequent]
    return fmc

groups_lemmas = [get_main_lemmas(g) for g in groups]
len_lemmas = np.array([len(g) for g in groups_lemmas])

## Indices of Interesting Groups

In [14]:
total_tweet_lens = np.array([sum(len(g) for g in group) for group in groups_claims])
avg_tweet_lens = total_tweet_lens / groups_lens

indices = ((groups_lens >= 10) & (groups_lens < 30) & (avg_tweet_lens > 140)).nonzero()[0]
# indices = ((groups_lens >= 30) & (groups_lens <= 100) & (avg_tweet_lens > 140)).nonzero()[0]
indices.sort()
len(indices)

184

## Filter Out those without a Lemma

In [15]:
wo = (len_lemmas[indices]==0).sum()
print(f'{wo} groups out of chosen {len(indices)} groups have no lemma')

31 groups out of chosen 184 groups have no lemma


In [16]:
sub_len_lemmas = len_lemmas[indices]
filtered_indices = indices[sub_len_lemmas > 0]

In [17]:
idx = filtered_indices[22]
cl = groups_claims[idx]
for i, c in enumerate(cl):
    if i==6: break
    print(c)

BIS varuje před destruktivní snahou autoritářských režimů, prezident místo snahy o zajištění bezpečnosti občanů zaútočí na BIS.   A poslancům je to jedno :-(
BIS informuje prezidenta o rozbití ruské zpravodajské sítě na našem území a o omezení vyhoštění ruských zpravodajců.  A prezident zaútočí na BIS, že nic nedělá.  Prezident dlouhodobě aktivně vystupuje proti zájmům ČR a ohrožuje bezpečnost země.   https://t.co/f695UuUbhj
Prezident svými vyjádřeními ohrožuje bezpečnost České republiky  Včera lživě zaútočil na BIS, i díky které jsme jednou z nejbezpečnějších zemí světa.  A koalice odmítá vznik Komise pro vyšetřování vlivu cizích mocností  Tak jsem napsal tweet. Ale neulevilo se mi a ničemu nepomůže
Člověk, který se tváří jako prezident, si nechal vyrobit kompro materiál na ředitele BIS vlastního státu, který slouží zemi a nechce otevřít dveře nepřátelským mocnostem. Hanebné.  Stojím za generálem Koudelkou, jemuž jde o bezpečnost země. Panu Zemanovi jde o přesný opak.
V normalní zemi 

In [18]:
permuter = np.random.permutation(len(filtered_indices))[:100]
permuter.sort()
shuffled_indices = filtered_indices[permuter]

for i, idx in zip(permuter, shuffled_indices):
    group = groups[idx]
    print(f'{i+1} & {groups_lens[idx]} &', *groups_lemmas[idx], '\\\\')

1 & 19 & zvíře SPD týrání sněmovna trest prosazovat návrh zákon \\
3 & 12 & SociologPetrHam TomasRazdik novinkycs Creative reflex Stones Company \\
4 & 28 & republika prezident převzít pověřovací listina nový velvyslanec hrad pražský \\
7 & 28 & miliarda regulace odhad samotný unijní stát HDP ročně \\
9 & 24 & Ukrajina \\
10 & 13 & SPD škola \\
11 & 28 & důchodce via \\
13 & 28 & svět třetí importovat stanout \\
14 & 10 & kvóta \\
18 & 19 & lumidek konv 333 astrophoto Austria 123 VaclavAgh SociologPetrHam Ness \\
19 & 17 & můj NATO bezpečnost \\
21 & 10 & republika druhý \\
23 & 11 & BIS prezident \\
25 & 13 & rasizmus \\
26 & 20 & prezident \\
27 & 11 & Dei martasturk TomasRazdik politikunatrik chyba syste AZahorakov \\
28 & 17 & odbor iDNEScs JMalacov AndrejBabis tripartita adamvojtechano \\
30 & 12 & prezident republika \\
31 & 20 & 1989 \\
32 & 11 & Izrael \\
33 & 16 & prezident pražský hrad natáčení pořad týden sledovat Barrandov \\
34 & 12 & Prima dít pořad Lána sledovat \\
35 & 

In [140]:
for i, idx in enumerate(filtered_indices):
    group = groups[idx]
    print(f'{i+1:3d} ({groups_lens[idx]:3d})', *groups_lemmas[idx])

  1 ( 41) smlouva mez zdanění daňový zamezení dvojí návrh
  2 ( 81) Brusel můj evropský český chtít ala
  3 ( 70) můj připomínat světový válek
  4 ( 94) prezident český republika Český ruský můj
  5 ( 82) republika prezident Zeman Miloš hrad pražský střed hodina jmenovat
  6 ( 52) důchod můj plat mluvit vláda
  7 ( 66) Euro euro ala přijetí odbor vláda
  8 ( 83) pirát PiratskaStran fašistický pirátský Antifa liberální
  9 ( 70) SPD můj hnutí Matteo Salvini
 10 ( 37) digitální danit firma platit řešení globální zdanění
 11 ( 85) prezident republika předseda Pan Zeman Miloš Lána pan
 12 ( 52) mzda růst odbor via muset plat
 13 ( 51) Český
 14 ( 83) volba volit jít moci můj kandidát
 15 ( 66) Ukrajinec odbor Ukrajina český via tripartita mpsvcs
 16 ( 56) evropský premiér český zájem aba Babiš
 17 ( 77) můj Německo německý děkovat
 18 ( 74) rakouský Rakousko můj Vídeň Austria vztah spolupráce
 19 ( 86) ošetřovné vláda rodič dítě
 20 ( 57) doprava všechen železnice dálnice správa silnice mi